In [1]:
import glob
import numpy as np
import operator
import time


In [2]:

d = 1792
K = 20
print("Started at %s" %time.ctime())

    


Started at Sun Apr 26 23:10:15 2020


In [3]:
allfiles = glob.glob('npz/*.npz')
X = []
file_name_mapping = {}
for i, file_name in enumerate(allfiles):
    file_name_mapping[i] = file_name
    X.append(np.loadtxt(file_name))

In [4]:
#generate query set, and random weights for testing
Q = np.random.randint(low=0, high=len(X), size=5)
queryFiles = [file_name_mapping[i] for i in Q]
W = [0.2, 0.3, 0.2, 0.5, 0.8]
Q = [X[i] for i in Q]



In [5]:
#calculate delta_ijl
delt = []
for l in range(0, d):
    delt_l = []
    for j in range(0, len(Q)):
        delt_j = []
        for i in range(0, len(X)):
            delta_value = W[j]*abs(X[i][l]-Q[j][l])
            delt_j.append((i, delta_value))
        delt_l.append(delt_j)
    delt.append(delt_l)
    


In [6]:
#"union" of the index list
KS_l_prime = []
for l in range(0, d):
    counter = {}
    for j in range(0, len(Q)):
        delt[l][j].sort(key = operator.itemgetter(1))
        for id, dist in [delt[l][j][k] for k in range(0, K)]:
            if id in counter:
                val = counter[id]
                counter[id] = (val[0]+1, val[1]+dist)
            else:
                counter[id] = (1, dist)
    KS_l_prime.append(counter)


In [7]:
#id of k loweset distance in dimension l
KS_l = []
for l in range(0, d):
    st = sorted(KS_l_prime[l].items(), key=lambda x: (-x[1][0], x[1][1]))
    KS_l.append(set([st[k][0] for k in range(0, K)]))

print(KS_l[0])
print(KS_l[1])
        

{4353, 7044, 13070, 9241, 14627, 23339, 18997, 4152, 59, 22844, 15040, 1857, 19395, 14408, 16207, 19666, 17502, 14949, 3960, 13946}
{9241, 7720, 23339, 23468, 16561, 18997, 22965, 15671, 15040, 4801, 7289, 15819, 14795, 21206, 16484, 2282, 22260, 3960, 12025, 4479}


In [8]:
#binary for each X_i, 1 if in KS_l of that dimension, 0 otherwise
B_i = []
for i in range(0, len(X)):
    b_i = []
    B_i.append([1 if i in KS_l[l] else 0 for l in range(0, d)])




In [9]:
def q_bar_l(l):
    nominator = 0
    denominator = 0
    for j in range(0, len(Q)):
        nominator += W[j] * Q[j][l]
        denominator += W[j]
    return nominator/denominator

#multiquery distance for each item
MSD_i = []
for i in range(0, len(X)):
    nominator = 0
    denominator = 0
    for l in range(0, d):
        if B_i[i][l] != 0:
            nominator += abs(X[i][l]-q_bar_l(l))
            denominator += 1
    denominator = denominator ** 2
    MSD_i.append((i, nominator/denominator if denominator > 0 else float('inf')))


    


In [10]:
st = sorted(MSD_i, key=lambda x: x[1])
MQKNN = [st[k] for k in range(0, K)]


In [11]:
print("QuerySet")
print(queryFiles)

print("MQKNN")
print([file_name_mapping[ans[0]] for ans in MQKNN])
print("Endedat at %s" %time.ctime())

QuerySet
['npz\\24_2656_6.npz', 'npz\\46_5020_2.npz', 'npz\\35_3874_2.npz', 'npz\\15_1727_5.npz', 'npz\\8_924_6.npz']
MQKNN
['npz\\26_2928_6.npz', 'npz\\0_0_4.npz', 'npz\\0_100_5.npz', 'npz\\0_101_2.npz', 'npz\\0_101_4.npz', 'npz\\0_0_1.npz', 'npz\\0_101_3.npz', 'npz\\0_102_4.npz', 'npz\\0_101_5.npz', 'npz\\0_100_1.npz', 'npz\\0_103_2.npz', 'npz\\0_0_5.npz', 'npz\\0_103_4.npz', 'npz\\0_102_1.npz', 'npz\\0_0_3.npz', 'npz\\0_103_5.npz', 'npz\\0_0_2.npz', 'npz\\0_102_5.npz', 'npz\\0_101_1.npz', 'npz\\0_103_3.npz']
Endedat at Mon Apr 27 00:14:19 2020


In [ ]:
W = [0.2, 0.3, 0.2, 0.5, 0.8]